In [ ]:
# Install relevant packages
# !pip install kraken

In [1]:
!kraken list

[04/16/25 14:19:17] WARNING  No metadata found for record            ]8;id=526110;file:///Users/siyuliang/anaconda3/lib/python3.11/site-packages/kraken/repo.py\repo.py]8;;\:]8;id=994427;file:///Users/siyuliang/anaconda3/lib/python3.11/site-packages/kraken/repo.py#264\264]8;;\
                             '10.5281/zenodo.15030337'.                         
[04/16/25 14:19:18] WARNING  No metadata found for record            ]8;id=528788;file:///Users/siyuliang/anaconda3/lib/python3.11/site-packages/kraken/repo.py\repo.py]8;;\:]8;id=673698;file:///Users/siyuliang/anaconda3/lib/python3.11/site-packages/kraken/repo.py#264\264]8;;\
                             '10.5281/zenodo.13862096'.                         
[04/16/25 14:19:22] WARNING  No metadata found for record            ]8;id=292619;file:///Users/siyuliang/anaconda3/lib/python3.11/site-packages/kraken/repo.py\repo.py]8;;\:]8;id=818305;file:///Users/siyuliang/anaconda3/lib/python3.11/site-packages/kraken/repo.py

In [31]:
!kraken show 10.5281/zenodo.7050270

name: 10.5281/zenodo.7050270

Printed Arabic-Script Base Model Trained on the OpenITI Corpus

Printed Arabic-Script Base Model Trained on the OpenITI Corpus

This is a text recognition model trained on the OpenITI dataset of printed
Arabic-script text available at [0] in its state of 2022-09-03. It encompasses
real world Arabic (~23k lines), Persian (~17k lines), Urdu (~11k lines), and
Ottoman Turkish (~7100 lines) material in a variety of typefaces augmented by a
synthetic data in the Tahoma (600 lines) typeface.

As the model is trained on a variety of languages and highly diverse typefaces
it is mostly intended as a base model for fine-tuning more specific models from
it. In line with this it has not been extensively verified or optimized.

The ground truth was lightly normalized to NFD but is otherwise untouched. 

[0]: https://github.com/OpenITI/arabic_print_data.git
scripts: Arab
alphabet: !"'()*+,-./0123456789:;<=>?ABCDEFGHIJKLMNOPRSTUVWX[\]_abcdefghijklmnoprstuvwxyz{|}«»،؎؛؟ءاب

In [ ]:
# Download the model
!kraken get 10.5281/zenodo.7050296

Processing ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 16.3/16.3 MB 0:00:00 0:00:1000:1000:09
Model name: arabic_best.mlmodel


In [ ]:
# Running on one sample image
!kraken -i ../data/v1.jpg ../output/kraken_arabic_best.txt segment -bl ocr -m arabic_best.mlmodel --text-direction horizontal-tb

Loading ANN /Users/siyuliang/anaconda3/lib/python3.11/site-packages/kraken/blla.mlmodel	✓
Loading ANN arabic_best.mlmodel	✓
Segmenting ../data/v1.jpg	✓
Processing ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 16/16 0:00:00 0:00:00
Writing recognition results for ../data/v1.jpg	✓


In [39]:
from pathlib import Path
from PIL import Image
from kraken import binarization, blla, pageseg, rpred
from kraken.lib import models

# ─── paths ────────────────────────────────────────────────────────────────
img_path  = Path("../data/v1.jpg")
out_path  = Path("../output/kraken_arabic_best.txt")
rec_model = Path("/Users/siyuliang//Library/Application Support/kraken/all_arabic_scripts.mlmodel")       # adjust if the file lives elsewhere

# ─── 1. load the image ────────────────────────────────────────────────────
im = Image.open(img_path)

# ─── 2. segment the page ─────────────────────────────────────────────────
# CLI flag  -bl  → baseline‑layout segmenter.  In the API that’s blla.segment.
# Kraken ships a *default* segmentation model, so model=None is fine.
# Valid text‑direction strings are "horizontal-lr" (L‑to‑R) or "horizontal-rl".
seg = blla.segment(im, text_direction="horizontal-lr")   # ← equivalent to -bl

# (If you ever want the legacy bounding‑box segmenter instead of -bl, do:)
# bw  = binarization.nlbin(im)          # binarize first
# seg = pageseg.segment(bw, text_direction="horizontal-lr")

# ─── 3. load the Arabic recogniser ────────────────────────────────────────
network = models.load_any(rec_model)

# ─── 4. run OCR line‑by‑line and save it ──────────────────────────────────
text_lines = [rec.prediction for rec in rpred.rpred(network, im, seg)]
out_path.parent.mkdir(parents=True, exist_ok=True)
out_path.write_text("\n".join(text_lines), encoding="utf‑8")
print("Done →", out_path)


/Users/siyuliang/anaconda3/lib/python3.11/site-packages/coremltools/models/model.py:561: RuntimeWarning: You will not be able to run predict() on this Core ML model. Underlying exception message was: Error compiling model: "compiler error: Error reading protobuf spec. validator error: Input MLMultiArray to neural networks must have dimension 1 (vector) or 3 (image-like arrays).".
  _warnings.warn(
/Users/siyuliang/anaconda3/lib/python3.11/site-packages/coremltools/models/model.py:561: RuntimeWarning: You will not be able to run predict() on this Core ML model. Underlying exception message was: Error compiling model: "compiler error: Error reading protobuf spec. validator error: Input MLMultiArray to neural networks must have dimension 1 (vector) or 3 (image-like arrays).".
  _warnings.warn(
/Users/siyuliang/anaconda3/lib/python3.11/site-packages/kraken/rpred.py:322: UserWarning: Using legacy polygon extractor, as the model was not trained with the new method. Please retrain your model 

Done → ../output/kraken_arabic_best.txt


In [ ]:
# Checking what models you have
# Need to be replaced with your own path
from pathlib import Path

# macOS default
kraken_dir = Path.home() / "Library/Application Support/kraken"

# Linux default would be:  Path.home() / ".local/share/kraken"

for p in kraken_dir.rglob("*.mlmodel"):
    print(p)


/Users/siyuliang/Library/Application Support/kraken/all_arabic_scripts.mlmodel
/Users/siyuliang/Library/Application Support/kraken/arabic_best.mlmodel


In [ ]:
# Full code to evaluate the Kraken Arabic model on a list of manuscript pages
# -*- coding: utf-8 -*-
"""
Evaluate the Kraken Arabic model on a list of manuscript pages,
save a TSV with CER/WER *and* write both the raw OCR output
and ground truth as .txt files.

Run from project root or from src/:
    $ python src/eval_kraken.py
"""

import json, random
from pathlib import Path
from collections import defaultdict, Counter
from PIL import Image
from kraken import blla, rpred
from kraken.lib import models

# ─── locate project root ──────────────────────────────────────────────────
try:                                   # running as script
    ROOT_DIR = Path(__file__).resolve().parent.parent   # …/ocr
except NameError:                      # running in notebook
    cwd = Path.cwd().resolve()
    ROOT_DIR = cwd if (cwd / "data").exists() else cwd.parent

# ─── PATHS & CONFIG ───────────────────────────────────────────────────────
JSON_PATH = ROOT_DIR / "data/jarring_manuscripts_data/jarring_manuscripts_structured.json"
IMG_ROOT  = ROOT_DIR / "data/jarring_manuscripts_data"          # prefix to local_image_path
OUT_DIR   = ROOT_DIR / "output"                                 # existing output folder
OUT_TSV   = OUT_DIR  / "kraken_eval.tsv"

REC_MODEL_PATH = Path.home() / "Library/Application Support/kraken/arabic_best.mlmodel"
TEXT_DIRECTION = "horizontal-rl"
SEED           = 42

# ═════════════════════════════════════════════════════════════════════════
# CONFIGURATION OPTIONS - Modify these to control which images to process
# ═════════════════════════════════════════════════════════════════════════

# Option 1: Use specific image paths (set to empty list to use PROCESS_ALL_IMAGES)
ALLOWED_IMAGES = [
    "images/Jarring_Prov_2/Jarring_Prov_2_004_r.Lanczos.800.65_4a.jpg",
    "images/Jarring_Prov_3/Jarring_Prov_3_fol_003_r.Lanczos.800.65_3a.jpg",
    "images/Jarring_Prov_4/Jarring_Prov_4_002_v.Lanczos.800.65_2b.jpg",
    "images/Jarring_Prov_5/Jarring_Prov_5_003_r.Lanczos.800.65_3a.jpg",
]

# Option 2: Process all images in JSON (set to True to override ALLOWED_IMAGES)
PROCESS_ALL_IMAGES = False

# ─── METRIC HELPERS ───────────────────────────────────────────────────────
def _lev(a, b):
    if len(a) < len(b):
        a, b = b, a
    prev = list(range(len(b) + 1))
    for i, ca in enumerate(a, 1):
        cur = [i]
        for j, cb in enumerate(b, 1):
            cur.append(min(cur[j-1] + 1,
                           prev[j]  + 1,
                           prev[j-1] + (ca != cb)))
        prev = cur
    return prev[-1]

def cer(ref, hyp):
    return 0.0 if not ref else _lev(ref.strip(), hyp.strip()) / len(ref)

def wer(ref, hyp):
    ref_t, hyp_t = ref.split(), hyp.split()
    return 0.0 if not ref_t else _lev(ref_t, hyp_t) / len(ref_t)

# ─── MAIN ─────────────────────────────────────────────────────────────────
def main():
    random.seed(SEED)
    OUT_DIR.mkdir(exist_ok=True, parents=True)

    # 1 ▸ load JSON
    with open(JSON_PATH, encoding="utf-8") as f:
        all_recs = json.load(f)

    # 2 ▸ filter records based on configuration
    if PROCESS_ALL_IMAGES:
        recs = all_recs
        print(f"Processing all {len(recs)} images from JSON")
    else:
        # Convert to set for faster lookups
        allowed_set = set(ALLOWED_IMAGES)
        recs = [r for r in all_recs if r["local_image_path"] in allowed_set]
        print(f"Processing {len(recs)} specified images")
    
    if not recs:
        raise RuntimeError("No specified images found in JSON.")

    # 3 ▸ bundle by manuscript for later aggregation
    by_ms = defaultdict(list)
    for r in recs:
        by_ms[r["manuscript_id"]].append(r)

    # 4 ▸ load recogniser
    network = models.load_any(REC_MODEL_PATH)

    # 5 ▸ OCR + metrics
    stats = defaultdict(lambda: Counter(lines=0, char_err=0, word_err=0))

    for rec in recs:
        ms_id      = rec["manuscript_id"]
        img_path   = IMG_ROOT / rec["local_image_path"]
        image      = Image.open(img_path)
        segments   = blla.segment(image, text_direction=TEXT_DIRECTION)
        preds      = [l.prediction for l in rpred.rpred(network, image, segments)]
        gold_lines = [ln["arabic_text"] for ln in rec["lines"]]

        # Create base filename for outputs
        base_filename = Path(rec["local_image_path"]).with_suffix("").name
        
        # save OCR text ----------------------------------------------------
        ocr_txt_path = OUT_DIR / f"{base_filename}.ocr.txt"
        ocr_txt_path.write_text("\n".join(preds), encoding="utf-8")
        
        # save ground truth text -------------------------------------------
        gt_txt_path = OUT_DIR / f"{base_filename}.gt.txt"
        gt_txt_path.write_text("\n".join(gold_lines), encoding="utf-8")

        # metrics ----------------------------------------------------------
        for ref, hyp in zip(gold_lines, preds):
            stats[ms_id]["lines"]    += 1
            stats[ms_id]["char_err"] += cer(ref, hyp) * len(ref)
            stats[ms_id]["word_err"] += wer(ref, hyp) * len(ref.split())

    # 6 ▸ aggregate & print/save table
    header = ["manuscript", "lines", "CER", "WER"]
    rows, total = [], Counter(lines=0, char_err=0, word_err=0)

    for ms_id, c in sorted(stats.items()):
        tot_chars_ms = sum(len(l["arabic_text"])
                           for r in recs if r["manuscript_id"] == ms_id
                           for l in r["lines"])
        rows.append((ms_id, c["lines"],
                     f"{c['char_err']/tot_chars_ms:.4f}",
                     f"{c['word_err']/c['lines']:.4f}"))
        total.update(c)

    tot_chars_all = sum(len(l["arabic_text"]) for r in recs for l in r["lines"])
    rows.append(("ALL", total["lines"],
                 f"{total['char_err']/tot_chars_all:.4f}",
                 f"{total['word_err']/total['lines']:.4f}"))

    w = [max(len(str(r[i])) for r in ([header] + rows)) for i in range(4)]
    fmt = "  ".join(f"{{:{k}}}" for k in w)

    print(fmt.format(*header))
    print("-"*sum(w)*2)
    for r in rows:
        print(fmt.format(*r))

    # TSV
    with OUT_TSV.open("w", encoding="utf-8") as f:
        f.write("\t".join(header) + "\n")
        for r in rows:
            f.write("\t".join(map(str, r)) + "\n")
    print("\n✓ Results:  ", OUT_TSV.relative_to(ROOT_DIR))
    print("✓ OCR txts: ", ", ".join(sorted(p.name for p in OUT_DIR.glob("*.ocr.txt"))))
    print("✓ GT txts:  ", ", ".join(sorted(p.name for p in OUT_DIR.glob("*.gt.txt"))))

if __name__ == "__main__":
    main()

Processing 4 specified images


/Users/siyuliang/anaconda3/lib/python3.11/site-packages/kraken/rpred.py:322: UserWarning: Using legacy polygon extractor, as the model was not trained with the new method. Please retrain your model to get speed improvement.
  warnings.warn('Using legacy polygon extractor, as the model was not trained with the new method. Please retrain your model to get speed improvement.')
Extracting line failed: Baseline length below minimum 5px
/Users/siyuliang/anaconda3/lib/python3.11/site-packages/coremltools/models/model.py:561: RuntimeWarning: You will not be able to run predict() on this Core ML model. Underlying exception message was: Error compiling model: "compiler error: Error reading protobuf spec. validator error: Input MLMultiArray to neural networks must have dimension 1 (vector) or 3 (image-like arrays).".
  _warnings.warn(
/Users/siyuliang/anaconda3/lib/python3.11/site-packages/kraken/rpred.py:322: UserWarning: Using legacy polygon extractor, as the model was not trained with the new 

manuscript      lines  CER     WER   
--------------------------------------------------------------
Jarring_Prov_2     10  0.9644  7.6000
Jarring_Prov_3      8  0.9109  7.1250
Jarring_Prov_4      9  0.8814  7.7778
Jarring_Prov_5      9  0.9256  7.8889
ALL                36  0.9222  7.6111

✓ Results:   output/kraken_eval.tsv
✓ OCR txts:  Jarring_Prov_2_004_r.Lanczos.800.65_4a.ocr.txt, Jarring_Prov_3_fol_003_r.Lanczos.800.65_3a.ocr.txt, Jarring_Prov_4_002_v.Lanczos.800.65_2b.ocr.txt, Jarring_Prov_5_003_r.Lanczos.800.65_3a.ocr.txt
✓ GT txts:   Jarring_Prov_2_004_r.Lanczos.800.65_4a.gt.txt, Jarring_Prov_3_fol_003_r.Lanczos.800.65_3a.gt.txt, Jarring_Prov_4_002_v.Lanczos.800.65_2b.gt.txt, Jarring_Prov_5_003_r.Lanczos.800.65_3a.gt.txt


In [24]:
!kraken get 10.5281/zenodo.7050270

Processing ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 16.3/16.3 MB 0:00:00 0:00:08m 0:00:08
Model name: all_arabic_scripts.mlmodel


In [ ]:
!kraken -i ../data/v1.jpg ../output/kraken_all_ara.txt segment -bl ocr -m all_arabic_scripts.mlmodel --text-direction horizontal-tb

Loading ANN /Users/siyuliang/anaconda3/lib/python3.11/site-packages/kraken/blla.mlmodel	✓
Loading ANN all_arabic_scripts.mlmodel	✓
Segmenting v1.jpg	✓
Processing ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 16/16 0:00:00 0:00:00
Writing recognition results for v1.jpg	✓
